In [2]:
# | results: hide
import pandas as pd
from datetime import datetime
import requests
import holoviews as hv
import locale

locale.setlocale(locale.LC_TIME, "de_DE.UTF-8")
hv.extension("bokeh", logo=False)

# Dynamischer Stromtarif vs. Fixpreistarif in Kombination mit Photovoltaikanlage

In unserer Eigentümergemeinschaft kann ich den Stromverbrauch der vier Wohneinheiten [digital in Echtzeit auslesen](https://github.com/grst/energymeter). Bereits im Vergangenen Jahr habe ich anhand der Verbrauchsdaten von 2024 unseren momentanen Stromtarif mit [Tado Hourly](https://energy.tado.com/) verglichen. Dabei kam ich zu dem Ergebnis, dass der dynamische Stromtarif auch ohne steuerbare Verbraucher  für alle Wohneinheiten (geringfügig) günstiger ist. 

In diesem Post gibt es ein Update mit den Daten von 2025. Zusätzlich berücksichtige ich in diesem Jahr den Strom, der von der Photovolatikanlage produziert wurde. Möglicherweise ist in Kombination mit PV der dynamische Stromtarif gar nicht mehr günstiger, weil der Strom vermutlich meist dann im Überfluss vorhanden ist, wenn man selber gerade Strom vom eigenen Dach bezieht. 

Die Rohdaten und das Python Notebook mit der Analyse stehen [auf GitHub zur Verfügung](https://github.com/grst/dynamischer-stromtarif). 

## Die Tarife

Es gibt eine Unzahl an unterschiedlichen Stromtarifen in Deutschland mit teilweise sehr unterschiedlichen
Preisstrukturen. Hier betrachte ich nur zwei Tarife: 

 * [AÜW Allgäustrom Basis](https://auew.de/privatkunden/strom/allgaeustrom-basis/), unser bisheriger Anbieter, Preisniveau für 2025
 * [Tado Hourly][]

Die Tarife gestalten sich wie folgt (alle Preise inkl. MwSt):

| Tarif | Arbeitspreis pro kWh | Grundpreis pro Monat |
| -- | -- | -- |
| AÜW | 34,62 ct (Strompreis + Steuern + Netzgebühr) | 15,68 EUR |
| tado | Epex Spot Day Ahead <br> + 19% MwSt <br> + 1,785ct (Aufschlag) <br> + 21,540 ct (Netzgebühr + Steuern) | 16,60 EUR |

: Verglichene Stromtarife {.striped .hover tbl-colwidths="[25,50,25]"}


Die Grundpreise sind sehr ähnlich, daher betrachte ich nur die Arbeitspreise. 

[Tado Hourly]: https://energy.tado.com/
[Tibber]: https://tibber.com/de

## Die Wohneinheiten

| Wohnung | Beschreibung |
| -- | -- |
|Wohnung 1 | Ganzjährig bewohnt von einer alleinstehenden Person. Ca. 25 Jahre alte Haushaltsgeräte. Aus historischen Gründen hängen diverse gemeinschaftliche Verbraucher (Treppenhaus, Heizung, ...) zusätzlich auf diesen Zähler |
|Wohnung 2 | Ganzjährig bewohnt von zwei Personen. Teilweise erneuerte Haushaltsgeräte. Eigene 3kWp Photovolatikanlage mit 6kWh Speicher hinter dem Zähler. |
|Wohnung 3 | Ganzjährig bewohnt von einer Familie mit zwei kleinen Kindern. Neubauwohnung mit modernen Haushaltsgeräten. |
|Wohnung 4 | Sporadisch genutzte Ferienwohnung |

: Verbrauchsprofile vier Wohnungen {.striped .hover tbl-colwidths="[25,75]"}

In [3]:
VAT = 1.19
YEAR = 2025
TADO_FIXED_FEE_PER_KWH = 0.21540  # EUR/kWh
AEUW_FEE_PER_KWH = 0.3462  # EUR/kWh
BASE_FEE_AUEW = 15.68

In [4]:
# Get hourly prices from awattar API
begin = datetime.fromisoformat("2020-01-01").timestamp()
end = datetime.now().timestamp()
# API endpoint URL
url = f"https://api.awattar.de/v1/marketdata?start={int(begin) * 1000}&end={int(end) * 1000}"

# Fetch JSON data from the API
response = requests.get(url)
data = response.json()

# Convert JSON data to Pandas DataFrame
hourly_price = pd.DataFrame(data["data"])

hourly_price["start"] = pd.to_datetime(hourly_price["start_timestamp"], unit="ms")
hourly_price["end"] = pd.to_datetime(hourly_price["end_timestamp"], unit="ms")
hourly_price["marketprice"], hourly_price["unit"] = hourly_price["marketprice"] / 1000, "EUR/kWh"
hourly_price["start_day"] = hourly_price["start"].dt.strftime(
    "%m-%d %H:%M:%S"
)  # day without year (for inter-year comparison)
hourly_price["real_price"] = hourly_price["marketprice"] * VAT + TADO_FIXED_FEE_PER_KWH

In [5]:
# Get recorded interval consumption and production data
all_interval = pd.read_csv("../data/2025_messwerte_pseudonymisiert_3600s_interval.csv", parse_dates=["time"])

In [6]:
consumption = all_interval.loc[lambda x: x["name"].isin(["Wohnung 1", "Wohnung 2", "Wohnung 3", "Wohnung 4"])]
production = all_interval.loc[
    lambda x: x["name"].isin(
        ["Sunny_Island_Batterie_entladen", "Sunny_Island_Netzbezug", "Sunny_Tripower_Gesamtertrag"]
    )
]

In [7]:
# Calcuate fraction of production by source (PV, Battery, Grid) for each hour
production_relative = (
    production.groupby("time")
    .apply(lambda x: x.assign(frac=x["Wh"] / x["Wh"].sum()))
    .reset_index(drop=True)
    .drop(columns=["Wh"])
    .rename(columns={"name": "source"})
)

/tmp/ipykernel_6179/1514857436.py:4: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(lambda x: x.assign(frac=x["Wh"] / x["Wh"].sum()))


In [8]:
# Calculate cost per hour, considering only the fraction that was drawn from the grid
cost_df = (
    consumption.merge(production_relative, on="time")
    .merge(hourly_price, left_on="time", right_on="start")
    .loc[lambda x: x["source"] == "Sunny_Island_Netzbezug"]
    .assign(Wh_grid=lambda x: x["Wh"] * x["frac"])
    .assign(
        # only energy from grid
        price_tado=lambda x: x["Wh_grid"] * x["real_price"] / 1000,
        price_auew=lambda x: x["Wh_grid"] * AEUW_FEE_PER_KWH / 1000,
        # all energy, theoretically, for comparison
        price_tado_no_pv = lambda x: x["Wh"] * x["real_price"] / 1000,
        prive_auew_no_pv = lambda x: x["Wh"] * AEUW_FEE_PER_KWH / 1000
    )
)

In [22]:
cost_df.groupby("name").apply(
    lambda x: x.set_index("time")[["Wh", "Wh_grid", "price_tado", "price_auew"]].resample("MS").sum()
).reset_index()

/tmp/ipykernel_509448/1955450972.py:1: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  cost_df.groupby("name").apply(


,name,time,Wh,Wh_grid,price_tado,price_auew
0,Wohnung 1,2025-01-01,323575.0,157001.931737,55.862797,54.354069
1,Wohnung 1,2025-02-01,278893.0,67060.757801,26.106008,23.216434
2,Wohnung 1,2025-03-01,299192.0,12021.814608,4.187036,4.161952
3,Wohnung 1,2025-04-01,283628.0,3279.581531,1.098568,1.135391
4,Wohnung 1,2025-05-01,285928.0,1937.328291,0.644602,0.670703
5,Wohnung 1,2025-06-01,292621.0,4140.163941,1.361569,1.433325
6,Wohnung 1,2025-07-01,322396.0,4210.461795,1.420826,1.457662
7,Wohnung 1,2025-08-01,305798.0,6550.389136,2.167932,2.267745
8,Wohnung 1,2025-09-01,312732.0,42095.095084,14.258459,14.573322
9,Wohnung 1,2025-10-01,339789.0,42627.596393,13.646989,14.757674


In [23]:
cost_df.groupby("name")[["Wh", "Wh_grid", "price_tado", "price_auew"]].sum()

,Wh,Wh_grid,price_tado,price_auew
name,,,,
Wohnung 1,3637097.0,596880.671131,208.911458,206.640088
Wohnung 2,1730636.0,558260.489724,195.963817,193.269782
Wohnung 3,1973203.0,327568.271493,120.350094,113.404136
Wohnung 4,583843.0,78888.247279,28.821280,27.311111
